In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import time
from scipy.special import lambertw
import matplotlib as mpl
import os
import glob


from noise_distribution import sample_laplace_radius, sample_psm_radius, sample_psm_radius_bounded
from util import haversine_distance, initial_bearing, destination_point

In [9]:
# # --- set global font to Times New Roman ---
# # ---- Camera-ready settings (avoid Type-3 fonts) ----
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.family'] = 'serif'




In [10]:



def planar_laplace_mechanism(lat: float, lon: float, epsilon: float, earth_radius: float = 6_371_000):
    r = sample_laplace_radius(epsilon)          # <-- exactly as you want
    theta = np.random.uniform(0, 2*np.pi)

    x = earth_radius * np.radians(lon)
    y = earth_radius * np.radians(lat)

    x_p = x + r * np.cos(theta)
    y_p = y + r * np.sin(theta)

    lon_p = np.degrees(x_p / earth_radius)
    lat_p = np.degrees(y_p / earth_radius)
    return lat_p, lon_p



In [11]:

def planar_staircase_mechanism(lat: float, lon: float, epsilon: float, delta: float = 1.0, earth_radius: float = 6_371_000):
    r = sample_psm_radius(epsilon, delta=delta)  # <-- same style
    theta = np.random.uniform(0, 2*np.pi)

    x = earth_radius * np.radians(lon)
    y = earth_radius * np.radians(lat)

    x_p = x + r * np.cos(theta)
    y_p = y + r * np.sin(theta)

    lon_p = np.degrees(x_p / earth_radius)
    lat_p = np.degrees(y_p / earth_radius)
    return lat_p, lon_p



def bounded_planar_staircase_mechanism(lat, lon, epsilon, r_b, earth_radius=6_371_000, delta_step=1.0):
    r = sample_psm_radius_bounded(epsilon, L=r_b, delta=delta_step)  # <= r_b by construction
    theta = np.random.uniform(0, 2*np.pi)

    x = earth_radius * np.radians(lon)
    y = earth_radius * np.radians(lat)

    x_p = x + r * np.cos(theta)
    y_p = y + r * np.sin(theta)

    lon_p = np.degrees(x_p / earth_radius)
    lat_p = np.degrees(y_p / earth_radius)
    return lat_p, lon_p


In [12]:
def apply_psmi_to_dataframe(
    df,
    epsilon,
    delta,
    r_b=50.0,
    thr_L=2.0,                # NEW: bound for threshold-noise (separate from r_b)
    earth_radius=6_371_000,
    use_noisy_threshold=True
):
    """
    Apply PSM-I to a trajectory DataFrame with 'latitude' and 'longitude'.

    - y1 = bounded PSM(x1) within r_b
    - z1 = PSM(y1)
    - For t>=2: y_t = Move(y_{t-1}, Dis(x_{t-1},x_t), Dir(x_{t-1},x_t))
    - Selective reporting: refresh z_t only if Dis(y_ref, y_t) >= threshold
    - threshold is delta or noisy (delta + bounded-PSM noise) once per session
    """
    n = len(df)
    y_lat = np.zeros(n)
    y_lon = np.zeros(n)
    z_lat = np.zeros(n)
    z_lon = np.zeros(n)

    # ----- threshold (sample once per file/session) -----
    threshold = float(delta)
    if use_noisy_threshold:
        noise = sample_psm_radius_bounded(epsilon, L=thr_L, delta=delta)
        threshold = float(delta + noise)

    # ----- init t=1 -----
    x1_lat, x1_lon = float(df.iloc[0]["latitude"]), float(df.iloc[0]["longitude"])

    y1_lat, y1_lon = bounded_planar_staircase_mechanism(
        x1_lat, x1_lon, epsilon, r_b,
        earth_radius=earth_radius,
        delta_step=delta
    )
    z1_lat, z1_lon = planar_staircase_mechanism(
        y1_lat, y1_lon, epsilon,
        delta=delta,
        earth_radius=earth_radius
    )

    y_lat[0], y_lon[0] = y1_lat, y1_lon
    z_lat[0], z_lon[0] = z1_lat, z1_lon

    y_ref_lat, y_ref_lon = y1_lat, y1_lon
    z_ref_lat, z_ref_lon = z1_lat, z1_lon

    # ----- main loop -----
    for t in range(1, n):
        x_prev_lat, x_prev_lon = float(df.iloc[t-1]["latitude"]), float(df.iloc[t-1]["longitude"])
        x_cur_lat,  x_cur_lon  = float(df.iloc[t]["latitude"]),   float(df.iloc[t]["longitude"])

        # displacement in true stream
        d_h = float(haversine_distance(x_prev_lat, x_prev_lon, x_cur_lat, x_cur_lon, earth_radius))
        theta = float(initial_bearing(x_prev_lat, x_prev_lon, x_cur_lat, x_cur_lon))

        # propagate intermediate by same displacement
        y_cur_lat, y_cur_lon = destination_point(y_lat[t-1], y_lon[t-1], d_h, theta, earth_radius)
        y_lat[t], y_lon[t] = y_cur_lat, y_cur_lon

        # selective reporting based on intermediate distance from last refresh
        d_y = float(haversine_distance(y_ref_lat, y_ref_lon, y_cur_lat, y_cur_lon, earth_radius))
        if d_y < threshold:
            z_lat[t], z_lon[t] = z_ref_lat, z_ref_lon
        else:
            z_new_lat, z_new_lon = planar_staircase_mechanism(
                y_cur_lat, y_cur_lon, epsilon,
                delta=delta,
                earth_radius=earth_radius
            )
            z_lat[t], z_lon[t] = z_new_lat, z_new_lon
            y_ref_lat, y_ref_lon = y_cur_lat, y_cur_lon
            z_ref_lat, z_ref_lon = z_new_lat, z_new_lon

    # true-to-released distances (utility)
    distances = haversine_distance(
        df["latitude"].values, df["longitude"].values,
        z_lat, z_lon,
        earth_radius
    )
    return y_lat, y_lon, z_lat, z_lon, distances


In [13]:
def batch_perturb_csv(
    input_folder,
    output_folder,
    mechanism="psm",
    epsilon=0.5,
    delta=5.0,
    r_b=50.0,
    thr_L=2.0,                 # NEW
    use_noisy_threshold=True,
    earth_radius=6_371_000
):
    """
    Perturb all CSVs in `input_folder` and save results in `output_folder`.

    mechanism: "psm", "plm", or "psmi"
    epsilon: privacy budget
    delta: base threshold for PSM-I selective reporting
    r_b: bounding radius for y1 initialization
    thr_L: bound for threshold-noise (separate from r_b)
    use_noisy_threshold: if False, threshold = delta (no privatization)
    """
    os.makedirs(output_folder, exist_ok=True)

    if mechanism == "psm":
        mech_fn = lambda lat, lon, eps: planar_staircase_mechanism(lat, lon, eps, delta=1.0, earth_radius=earth_radius)
    elif mechanism == "plm":
        mech_fn = lambda lat, lon, eps: planar_laplace_mechanism(lat, lon, eps, earth_radius=earth_radius)
    else:
        mech_fn = None

    for filepath in glob.glob(os.path.join(input_folder, "*.csv")):
        df = pd.read_csv(filepath)

        if not {"latitude", "longitude"}.issubset(df.columns):
            print(f"Skipping {filepath}: missing latitude/longitude")
            continue

        if mechanism in ("psm", "plm"):
            perturb = df.apply(
                lambda row: mech_fn(row["latitude"], row["longitude"], epsilon),
                axis=1, result_type="expand"
            )
            perturb.columns = ["perturbed_latitude", "perturbed_longitude"]
            df = pd.concat([df, perturb], axis=1)

            df["distance"] = haversine_distance(
                df["latitude"].values, df["longitude"].values,
                df["perturbed_latitude"].values, df["perturbed_longitude"].values,
                earth_radius
            )

        elif mechanism == "psmi":
            y_lat, y_lon, z_lat, z_lon, distances = apply_psmi_to_dataframe(
                df,
                epsilon=epsilon,
                delta=delta,
                r_b=r_b,
                thr_L=thr_L,
                earth_radius=earth_radius,
                use_noisy_threshold=use_noisy_threshold
            )
            df["intermediate_latitude"]  = y_lat
            df["intermediate_longitude"] = y_lon
            df["perturbed_latitude"]     = z_lat
            df["perturbed_longitude"]    = z_lon
            df["distance"]               = distances

        else:
            raise ValueError("mechanism must be one of: 'psm', 'plm', 'psmi'")

        out_path = os.path.join(output_folder, os.path.basename(filepath))
        df.to_csv(out_path, index=False)


In [14]:
# =============================================================================
# RUN EXPERIMENT
# =============================================================================
if __name__ == "__main__":
    epsilons = [0.1]
    input_folder = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\original\geotrace\security"

    base_dirs = {
        "psm":  r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\psm",
        "plm":  r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\plm",
        "psmi": r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\psmi"
    }

    for mechanism, base_out in base_dirs.items():
        for eps in epsilons:
            suffix = f"eps{int(eps*10):02d}" if eps < 1 else f"eps{int(eps)}"
            out_folder = os.path.join(base_out, suffix)
            print(f"> {mechanism.upper()}  ε={eps} → {out_folder}")

            batch_perturb_csv(
                input_folder=input_folder,
                output_folder=out_folder,
                mechanism=mechanism,
                epsilon=eps,
                delta=3.0,
                r_b=50.0,
                thr_L=5.0,                 
                use_noisy_threshold=True
            )


> PSM  ε=0.1 → C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\psm\eps01
> PLM  ε=0.1 → C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\plm\eps01
> PSMI  ε=0.1 → C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\psmi\eps01
